In [1]:
from youtube_transcript_api import YouTubeTranscriptApi

In [2]:
video_id = 'rMq63r3zi4c' 
ytt_api  = YouTubeTranscriptApi()

In [3]:
transcript = ytt_api.fetch(video_id)

In [6]:
transcript[11]

FetchedTranscriptSnippet(text='think I should already update this', start=26.84, duration=4.2)

In [7]:
def format_timestamp(seconds: float) -> str:
    total_seconds = int(seconds)
    hours, remainder = divmod(total_seconds, 3600)
    minutes, secs = divmod(remainder, 60)

    if hours == 0:
        return f"{minutes}:{secs:02}"
    return f"{hours}:{minutes:02}:{secs:02}"

def make_subtitles(transcript) -> str:
    lines = []

    for entry in transcript:
        ts = format_timestamp(entry.start)
        text = entry.text.replace('\n', ' ')
        lines.append(ts + ' ' + text)

    return '\n'.join(lines)

In [8]:
subtitles = make_subtitles(transcript)
print(subtitles[:500])

0:00 hi everyone Welcome to our event this
0:03 event is brought to you by data do club
0:05 which is a community of people who love
0:07 data we have weekly events well lately
0:10 they're not so weekly but we're getting
0:13 back on track and this week we will
0:15 actually have two events anyways so if
0:18 you want to find out about all the
0:20 events we have in our pipeline which is
0:22 currently I think two there's a link in
0:24 the description go check it out and I
0:26 think I should 
